In [5]:
import pdfplumber
import re

In [ ]:
loc_1 = r"/Users/azerty/Downloads/invoice1-word-example.pdf"

In [33]:
with pdfplumber.open(loc_1) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()

m = re.search(r'TOTAL\s*\$([\d,\.]+)', text, re.IGNORECASE)
total_cost = m.group(1) if m else None
print(total_cost)

source = [
r'INVOICE\s*(?:#|No\.?|Number|Num|Nº)?\s*[:#-]?\s*([A-Z0-9\-]+)',
r'INV(?:O?ICE)?\s*(?:#|No\.?|Number)?\s*[:#-]?\s*([A-Z0-9\-]+)',
r'Invoice\s+No\.?\s*[:#-]?\s*([A-Z0-9\-]+)',
r'Invoice\s+number\s*[:#-]?\s*([A-Z0-9\-]+)',
r'\b([A-Z]{2,5}-\d{3,6})\b',
r'\b([0-9]{4,12})\b',
]


100.00


In [34]:
import re, json
# Invoice number patterns (try many formats)
patterns = [
    r'INVOICE\s*(?:#|No\.?|Number|Num|Nº)?\s*[:#-]?\s*([A-Z0-9\-]+)',
    r'INV(?:O?ICE)?\s*(?:#|No\.?|Number)?\s*[:#-]?\s*([A-Z0-9\-]+)',
    r'Invoice\s+No\.?\s*[:#-]?\s*([A-Z0-9\-]+)',
    r'Invoice\s+number\s*[:#-]?\s*([A-Z0-9\-]+)',
    r'\b([A-Z]{2,5}-\d{3,6})\b',
    r'\b([0-9]{4,12})\b',
]
invoice_number = None
for p in patterns:
    m = re.search(p, text, re.IGNORECASE)
    if m:
        invoice_number = m.group(1)
        break

# Helper: first two words after a trigger
def first_two_after(trigger, text, n=2):
    m = re.search(r'\b' + re.escape(trigger) + r'\b', text, re.IGNORECASE)
    if not m:
        return None
    rest = text[m.end():].lstrip()
    rest = re.sub(r'\s+', ' ', rest)
    rest = re.sub(r'^[\:\.,\-\s]+', '', rest)
    parts = re.findall(r'\S+', rest)
    return ' '.join(parts[:n]) if parts else None

bill_to = first_two_after('BILLED TO', text) or first_two_after('BILL TO', text)

# Total amount: try multiple common labels
total = None
total_patterns = [r'TOTAL\s*\$([\d,\.]+)', r'Amount Due[:\s]*\$?([\d,\.]+)', r'Total Due[:\s]*\$?([\d,\.]+)']
for p in total_patterns:
    m = re.search(p, text, re.IGNORECASE)
    if m:
        total = m.group(1)
        break

# Item description: capture lines under header until blank line
m = re.search(r'ITEM DESCRIPTION.*?AMOUNT\s*\n(.+?)(?:\n\s*\n|\Z)', text, re.IGNORECASE | re.DOTALL)
item_description = m.group(1).strip() if m else None
if item_description:
    item_description = ' '.join([ln.strip() for ln in item_description.split('\n') if ln.strip()])

result = {
    'invoice_number': invoice_number,
    'bill_to': bill_to,
    'total_cost': total,
    'item_description': item_description
}
print(json.dumps(result, ensure_ascii=False))

{"invoice_number": "67", "bill_to": "John Smith", "total_cost": "100.00", "item_description": null}
